In [ ]:
#* Cargamos las paqueterias necesarias
import pandas as pd
import boto3
import awswrangler as wr
session = boto3.Session(profile_name='arquitectura')
s3 = session.client('s3')

#* Agregamos las especificaciones necesarias
database_name = "econ"

In [ ]:
#* Tabla de tipo de cambio
query = '''
    -- Create tipo_cambio table
CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tipo_de_cambio` (
    `timestamp` date,
    `tipo_de_cambio` bigint
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://itam-analytics-anayeli/raw/'
TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

wr.athena.read_sql_query(query, database=database_name, ctas_approach=False, boto3_session=session)

In [ ]:
#* Tabla de la tasa de interes
query = '''
-- Create tasa_interes table
CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`tasa_interes` (
    `timestamp` date,
    `tasa_interes` bigint
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://itam-analytics-anayeli/raw/'
TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

wr.athena.read_sql_query(query, database=database_name, ctas_approach=False, boto3_session=session)

In [ ]:
#* Tabla de la inflacion
query = '''
-- Create tasa_interes table
CREATE EXTERNAL TABLE IF NOT EXISTS `econ`.`inflacion` (
    `timestamp` date,
    `inflacion` bigint
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://itam-analytics-anayeli/raw/'
TBLPROPERTIES ('classification' = 'csv', "skip.header.line.count"="1");
'''

wr.athena.read_sql_query(query, database=database_name, ctas_approach=False, boto3_session=session)

In [8]:
#* Generamos un join con las tres tablas que creamos y lo guardamos en un dataframe
query = '''
SELECT COALESCE(tc.timestamp, ti.timestamp, inf.timestamp) AS date,
       tc.tipo_de_cambio,
       ti.tasa_interes,
       inf.inflacion
FROM econ.tipo_de_cambio tc
INNER JOIN econ.tasa_interes ti on tc.timestamp=ti.timestamp
INNER JOIN econ.inflacion as inf ON COALESCE(tc.timestamp, ti.timestamp) = inf.timestamp;
'''

data = wr.athena.read_sql_query(query, database=database_name, ctas_approach=False, boto3_session=session)